In [1]:
import numpy as np


In [2]:
# 概率转移矩阵 
# j列和为1, 表示转移到各个行i的概率(出度的倒数)
# 随机矩阵 每行或每列的值的和为1
M = np.array([
    [0, 0.5, 1, 0],
    [1/3, 0, 0, 1/2],
    [1/3, 0, 0, 1/2],
    [1/3, 1/2, 0, 0]])
M

array([[0.        , 0.5       , 1.        , 0.        ],
       [0.33333333, 0.        , 0.        , 0.5       ],
       [0.33333333, 0.        , 0.        , 0.5       ],
       [0.33333333, 0.5       , 0.        , 0.        ]])

In [35]:
lambda_, u = np.linalg.eig(M)


In [36]:
lambda_

array([-0.5,  1. ,  0. , -0.5])

In [37]:
u  # 一列为一个特征向量

array([[-0.8660254 ,  0.65465367,  0.70710678,  0.8660254 ],
       [ 0.28867513,  0.43643578, -0.47140452, -0.28867513],
       [ 0.28867513,  0.43643578,  0.23570226, -0.28867513],
       [ 0.28867513,  0.43643578, -0.47140452, -0.28867513]])

In [46]:
u0 = np.zeros_like(u)
u0[:, 1] = u[:, 1]
u0

array([[0.        , 0.65465367, 0.        , 0.        ],
       [0.        , 0.43643578, 0.        , 0.        ],
       [0.        , 0.43643578, 0.        , 0.        ],
       [0.        , 0.43643578, 0.        , 0.        ]])

In [56]:
np.linalg.inv(u)[:, 1]

array([-6.00479950e+15,  5.09175077e-01, -1.41421356e+00, -6.00479950e+15])

In [55]:
 u0 @ np.linalg.inv(u)[:, 1]

array([0.33333333, 0.22222222, 0.22222222, 0.22222222])

In [3]:
R0 = np.ones(4).reshape(4, 1) / 4
R0

array([[0.25],
       [0.25],
       [0.25],
       [0.25]])

In [15]:
def transform(init, trans, max_iter):
    max_iter
    temp = init
    for _ in range(max_iter):
        res = trans @ temp
        temp = res
    return res
transform(R0, M, 5)

array([[0.3359375 ],
       [0.22135417],
       [0.22135417],
       [0.22135417]])

In [16]:
transform(R0, M, 10)

array([[0.33325195],
       [0.22224935],
       [0.22224935],
       [0.22224935]])

In [17]:
# 不是随机矩阵, 有列为0
M2 = np.array([
    [0, 0.5, 0, 0],
    [1/3, 0, 0, 1/2],
    [1/3, 0, 0, 1/2],
    [1/3, 1/2, 0, 0]])
transform(R0, M2, 3)

array([[0.07291667],
       [0.10763889],
       [0.10763889],
       [0.10763889]])

In [18]:
transform(R0, M2, 5)

array([[0.0390625 ],
       [0.05700231],
       [0.05700231],
       [0.05700231]])

In [21]:
transform(R0, M2, 15)  # 各个节点的概率为0

array([[0.00165099],
       [0.00240619],
       [0.00240619],
       [0.00240619]])

# PageRank的一般定义

$$
R = dMR + \frac {1-d}{n}\bf1
$$
其中 $d (0 \leq d \leq 1)$是系数, 称为`阻尼因子`(damping factor), $R$是n为向量, $\bf1$是所有分量为1的n维向量

In [22]:
one = np.ones_like(R0)
one

array([[1.],
       [1.],
       [1.],
       [1.]])

In [30]:
def transform2(init, trans, d, max_iter):
    one = np.ones_like(init) / len(init)
    temp = init
    for _ in range(max_iter):
        temp = d * trans @ temp + (1 - d) * one
    return temp


In [31]:
transform2(R0, M, 0.8, 20)

array([[0.32142857],
       [0.22619048],
       [0.22619048],
       [0.22619048]])

In [32]:
transform2(R0, M2, 0.8, 20)

array([[0.10135339],
       [0.12838135],
       [0.12838135],
       [0.12838135]])

In [33]:
M3 = np.array([
    [0, 0.5, 0, 0],
    [1/3, 0, 0, 1/2],
    [1/3, 0, 1, 1/2],
    [1/3, 1/2, 0, 0]])
transform2(R0, M3, 0.8, 20)

array([[0.10135339],
       [0.12838135],
       [0.64188392],
       [0.12838135]])

In [107]:
def transform_power(x0, trans, d, max_iter, tol):
    E = np.ones_like(trans)
    x_last = x0
    for i in range(max_iter):
        A = d * trans + (1-d) / len(x0) * E
        y = A @ x_last
        # 绝对值最大的
        x = y / np.linalg.norm(y, ord=np.inf)
        if np.linalg.norm(x - x_last) < tol:
            print(i)
            break
        x_last = x
    
    x = x / np.sum(x)
    return x

In [108]:
M = np.array([
    [0, 0, 1],
    [1/2, 0, 0],
    [1/2, 1, 0]
])
x0 = np.ones((3, 1))
pagerank = transform_power(x0, M, 0.85, 22, 0.001)
pagerank

14


array([[0.3878706 ],
       [0.21474832],
       [0.39738108]])

In [94]:
# 代数方法矩阵求逆
def transform_algebra(trans, d):
    n = len(trans)
    I = np.eye(n)
    one = np.ones((n, 1))
    R = np.linalg.inv(I - d*trans) @((1-d) / n * one)
    return R

In [95]:
pagerank = transform_algebra(M, 0.85)
pagerank

array([[0.38778971],
       [0.21481063],
       [0.39739966]])

可供参考

https://blog.csdn.net/weixin_43378396/article/details/90322422

https://blog.csdn.net/hguisu/article/details/7996185